In [1]:
# Importing libraries for out RNN and for our corpus pre-processing

import tensorflow as tf

import numpy as np
import os
import time
import glob

2023-01-11 15:43:40.484572: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 15:43:41.017655: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-11 15:43:41.017728: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-11 15:43:44.782484: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
# Making our 13 txt wikicréole files being just one big string object

all_texts = ""

for text in glob.glob('../corpus/*.txt'):
    try:
        doc = open(text, 'rb').read().decode(encoding='utf-8')
        all_texts += str(doc)
    except:
        continue

In [3]:
# Take a look at the first 250 characters in all_texts
print(all_texts[:250])

{{Redirect2|Région parisienne|Île de France|homonymie=Île-de-France (homonymie)}}
{{Voir homonymes|IDF}}
{{Infobox Région de France
 | nom                       = Île-de-France
 | logo                      = Région_Île-de-France_(logo).svg
 | siège d


In [4]:
# The unique characters in the file
vocab = sorted(set(all_texts))
print(f'{len(vocab)} unique characters')

201 unique characters


In [5]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

2023-01-11 15:43:48.492932: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-11 15:43:48.493014: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-11 15:43:48.493092: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (diego-Inspiron-5370): /proc/driver/nvidia/version does not exist
2023-01-11 15:43:48.496033: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [6]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [7]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[65, 66, 67, 68, 69, 70, 71], [88, 89, 90]]>

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
# Back to string

tf.strings.reduce_join(chars, axis=-1).numpy()

def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


In [10]:
all_ids = ids_from_chars(tf.strings.unicode_split(all_texts, 'UTF-8'))
all_ids

<tf.Tensor: shape=(2050503,), dtype=int64, numpy=array([91, 91, 53, ..., 63,  2,  2])>

In [11]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [12]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

{
{
R
e
d
i
r
e
c
t


In [13]:
seq_length = 150

In [14]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq)) 

tf.Tensor(
[b'{' b'{' b'R' b'e' b'd' b'i' b'r' b'e' b'c' b't' b'2' b'|' b'R'
 b'\xc3\xa9' b'g' b'i' b'o' b'n' b' ' b'p' b'a' b'r' b'i' b's' b'i' b'e'
 b'n' b'n' b'e' b'|' b'\xc3\x8e' b'l' b'e' b' ' b'd' b'e' b' ' b'F' b'r'
 b'a' b'n' b'c' b'e' b'|' b'h' b'o' b'm' b'o' b'n' b'y' b'm' b'i' b'e'
 b'=' b'\xc3\x8e' b'l' b'e' b'-' b'd' b'e' b'-' b'F' b'r' b'a' b'n' b'c'
 b'e' b' ' b'(' b'h' b'o' b'm' b'o' b'n' b'y' b'm' b'i' b'e' b')' b'}'
 b'}' b'\n' b'{' b'{' b'V' b'o' b'i' b'r' b' ' b'h' b'o' b'm' b'o' b'n'
 b'y' b'm' b'e' b's' b'|' b'I' b'D' b'F' b'}' b'}' b'\n' b'{' b'{' b'I'
 b'n' b'f' b'o' b'b' b'o' b'x' b' ' b'R' b'\xc3\xa9' b'g' b'i' b'o' b'n'
 b' ' b'd' b'e' b' ' b'F' b'r' b'a' b'n' b'c' b'e' b'\n' b' ' b'|' b' '
 b'n' b'o' b'm' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' '], shape=(151,), dtype=string)


In [15]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'{{Redirect2|R\xc3\xa9gion parisienne|\xc3\x8ele de France|homonymie=\xc3\x8ele-de-France (homonymie)}}\n{{Voir homonymes|IDF}}\n{{Infobox R\xc3\xa9gion de France\n | nom             '
b'          = \xc3\x8ele-de-France\n | logo                      = R\xc3\xa9gion_\xc3\x8ele-de-France_(logo).svg\n | si\xc3\xa8ge du conseil r\xc3\xa9gional = [[Saint-Ouen-sur-Seine]]\n | pr\xc3\xa9'
b'fecture                = [[Paris]]\n | d\xc3\xa9partements              = [[Paris]] (75)<br />[[Seine-et-Marne]] (77)<br />[[Yvelines]] (78)<br />[[Essonne (d\xc3\xa9'
b"partement)|Essonne]] (91)<br />[[Hauts-de-Seine]] (92)<br />[[Seine-Saint-Denis]] (93)<br />[[Val-de-Marne]] (94)<br />[[Val-d'Oise]] (95)\n | arrondiss"
b"ements           = 25\n | cantons                   = 155\n | communes                  = 1268\n | conseil r\xc3\xa9gional          = [[Conseil r\xc3\xa9gional d'\xc3\x8ele-de"


In [16]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [17]:
# Exemple

split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [18]:
dataset = sequences.map(split_input_target)

In [19]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'{{Redirect2|R\xc3\xa9gion parisienne|\xc3\x8ele de France|homonymie=\xc3\x8ele-de-France (homonymie)}}\n{{Voir homonymes|IDF}}\n{{Infobox R\xc3\xa9gion de France\n | nom            '
Target: b'{Redirect2|R\xc3\xa9gion parisienne|\xc3\x8ele de France|homonymie=\xc3\x8ele-de-France (homonymie)}}\n{{Voir homonymes|IDF}}\n{{Infobox R\xc3\xa9gion de France\n | nom             '


In [20]:
# Batch size
BATCH_SIZE = 30

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 1000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(30, 150), dtype=tf.int64, name=None), TensorSpec(shape=(30, 150), dtype=tf.int64, name=None))>

In [21]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 100

# Number of RNN units
rnn_units = 100


In [22]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [23]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(30, 150, 202) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  20200     
                                                                 
 gru (GRU)                   multiple                  60600     
                                                                 
 dense (Dense)               multiple                  20402     
                                                                 
Total params: 101,202
Trainable params: 101,202
Non-trainable params: 0
_________________________________________________________________


In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [27]:
sampled_indices

array([116, 147, 186, 118,  67, 175, 142, 180,  58, 110, 187, 113,  35,
       115, 131,  41,   6, 193,  66, 165,   1, 105, 132,  55, 200,  77,
        71, 176, 198,   8,  62,  30, 105, 119,  13, 101,   8, 136,  89,
       183,  76, 129, 110,  79,  81,  10,  63, 193,  52, 133, 188, 153,
       136, 104,  70,  17, 198, 175,  75,  68, 195,   9,  59, 174,  61,
       103, 199, 185,  77,  24,  89, 185,   8, 180, 162,  52,  73, 131,
       194,   6, 192, 153, 125, 172,   3, 195, 112, 146, 154, 139, 186,
       128,  24, 160, 124, 138, 156, 113,  91,  48,  11, 106,  13,  15,
       181, 112, 180,  85,  36,  10,  57, 142, 155,  86, 162, 162, 182,
       160, 193, 139, 173, 176, 131, 118, 134,  36, 122,  37, 194, 100,
       183,   2,  24, 174, 115, 161,  82,  95,   7, 200,  22, 150, 191,
        75, 187,   5, 129,  42,  20,  86])

In [28]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"dr\xc3\xa9 Lesort, Pierre Bernus, Maurice-Pierre Boy\xc3\xa9, Monique Richard | titre=Visages de l'\xc3\x8ele-de-France | lieu=Paris | \xc3\xa9diteur=Horizons de France | ann\xc3\xa9e=1"

Next Char Predictions:
 b'\xc3\x97\xc9\x91\xe2\x80\x91\xc3\xa0c\xcf\x84\xc5\x8b\xe1\xba\xbfW\xc3\x89\xe2\x80\x93\xc3\x92@\xc3\x94\xc3\xafF#\xe2\x80\xa2b\xce\xb5\t\xc2\xbb\xc3\xb0T\xe2\xa9\xbdmg\xcf\x85\xe2\x88\x92%[;\xc2\xbb\xc3\xa1*\xc2\xb1%\xc3\xb4y\xe1\xbd\xb6l\xc3\xad\xc3\x89oq\']\xe2\x80\xa2Q\xc3\xb1\xe2\x80\x94\xca\xb3\xc3\xb4\xc2\xbaf.\xe2\x88\x92\xcf\x84kd\xe2\x80\xbf&X\xcf\x83Z\xc2\xb7\xe2\x8b\x86\xe2\x80\x89m5y\xe2\x80\x89%\xe1\xba\xbf\xce\xaeQi\xc3\xaf\xe2\x80\xa6#\xe2\x80\xa0\xca\xb3\xc3\xa9\xcf\x81 \xe2\x80\xbf\xc3\x8e\xc8\x9b\xcb\x88\xc3\xba\xe2\x80\x91\xc3\xac5\xce\x9c\xc3\xa8\xc3\xb9\xcc\x83\xc3\x92{M(\xc3\x80*,\xe1\xbb\xab\xc3\x8e\xe1\xba\xbfuA\'V\xc5\x8b\xcb\x8cv\xce\xae\xce\xae\xe1\xbc\xb1\xce\x9c\xe2\x80\xa2\xc3\xba\xcf\x82\xcf\x85\xc3\xaf\xc3\xa0\xc3\xb2A\xc3\xa6B\xe2\x8

In [29]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [30]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (30, 150, 202)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(5.309771, shape=(), dtype=float32)


In [31]:
tf.exp(example_batch_mean_loss).numpy()

202.30391

In [32]:
model.compile(optimizer='adam', loss=loss)

In [33]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [34]:
EPOCHS = 50

In [35]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
452/452 [==============================] - 109s 228ms/step - loss: 3.0685
Epoch 2/50
452/452 [==============================] - 103s 227ms/step - loss: 2.2713
Epoch 3/50
452/452 [==============================] - 102s 223ms/step - loss: 1.9887
Epoch 4/50
452/452 [==============================] - 101s 223ms/step - loss: 1.8417
Epoch 5/50
452/452 [==============================] - 90s 199ms/step - loss: 1.7542
Epoch 6/50
452/452 [==============================] - 87s 192ms/step - loss: 1.6961
Epoch 7/50
452/452 [==============================] - 88s 192ms/step - loss: 1.6551
Epoch 8/50
452/452 [==============================] - 87s 192ms/step - loss: 1.6238
Epoch 9/50
452/452 [==============================] - 95s 210ms/step - loss: 1.6002
Epoch 10/50
398/452 [=========================>....] - ETA: 11s - loss: 1.5918

KeyboardInterrupt: 

In [ ]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['{'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))